In [2]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error
import numpy as np
import matplotlib.pyplot as plt
import ldpc.codes
from ldpc import BpOsdDecoder
from qiskit.transpiler import CouplingMap
simulator = AerSimulator(method='density_matrix')

In [3]:
def generate_unicycle_ldpc(m, n):
    """
    Generate a unicycle LDPC code.
    
    Parameters:
        m (int): Number of rows (check bits).
        n (int): Number of columns (code bits).
        
    Returns:
        Hx (ndarray): LDPC parity-check matrix for X errors.
        Hz (ndarray): LDPC parity-check matrix for Z errors.
    """
    H = np.random.randint(0, 2, (m, n))
    Hx = H
    Hz = H[:, ::-1]  # Reverse the columns for unicycle codes
    return Hx, Hz

def rand_bin_array(K, N):
    """
    Generate a random binary array with K ones and (N-K) zeros.
    
    Parameters:
        K (int): Number of ones.
        N (int): Total length of the array.
        
    Returns:
        arr (ndarray): Shuffled binary array.
    """
    arr = np.zeros(N)
    arr[:K] = 1
    np.random.shuffle(arr)
    return arr

def cyclic_shift_matrix(arr):
    """
    Generate a cyclically shifted matrix from a binary array.
    
    Parameters:
        arr (ndarray): Binary array.
        
    Returns:
        matrix (ndarray): Cyclically shifted matrix.
    """
    N = len(arr)
    matrix = np.zeros((N, N), dtype=arr.dtype)
    for i in range(N):
        matrix[i] = np.roll(arr, i)
    return matrix

def generate_bicycle_ldpc_1(n, m, k):
    """
    Generate a Bicycle LDPC code.
    
    Parameters:
        n (int): Number of columns in the submatrix.
        m (int): Number of rows in the submatrix.
        k (int): Number of ones in the initial row.
        
    Returns:
        H (ndarray): Bicycle LDPC parity-check matrix.
    """
    n_sub = int(n / 2)
    row = rand_bin_array(k, n_sub)
    c = cyclic_shift_matrix(row)
    return np.concatenate((c, c.T), axis=1)


In [4]:
# Predefined LDPC parity-check matrices for X and Z errors
Hx = np.array([[1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
               [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0],
               [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
               [1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0],
               [0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
               [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1]])

Hz = np.array([[1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
               [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0],
               [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
               [1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0],
               [0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
               [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1]])

# Alternatively, generate Bicycle LDPC matrices
# Hx = generate_bicycle_ldpc_1(n=12, m=3, k=3)
# Hz = Hx.copy()

# Verify self-orthogonality: Hx * Hz^T = 0 mod 2
check_orthogonality = np.mod(np.dot(Hx, Hz.T), 2)
print("Hx:\n", Hx)
print("\nHz:\n", Hz)
print("\nHx * Hz^T mod 2:\n", check_orthogonality)
print("\nRank of Hx:", np.linalg.matrix_rank(Hx))


Hx:
 [[1 0 0 1 0 1 1 1 0 1 0 0]
 [1 1 0 0 1 0 0 1 1 0 1 0]
 [0 1 1 0 0 1 0 0 1 1 0 1]
 [1 0 1 1 0 0 1 0 0 1 1 0]
 [0 1 0 1 1 0 0 1 0 0 1 1]
 [0 0 1 0 1 1 1 0 1 0 0 1]]

Hz:
 [[1 0 0 1 0 1 1 1 0 1 0 0]
 [1 1 0 0 1 0 0 1 1 0 1 0]
 [0 1 1 0 0 1 0 0 1 1 0 1]
 [1 0 1 1 0 0 1 0 0 1 1 0]
 [0 1 0 1 1 0 0 1 0 0 1 1]
 [0 0 1 0 1 1 1 0 1 0 0 1]]

Hx * Hz^T mod 2:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]

Rank of Hx: 6


In [5]:
# Create a zero matrix of the same shape as Hx
zeros = np.zeros(Hx.shape)

# Concatenate Hx and Hz to form the full parity-check matrix H0
H0 = np.concatenate((
    np.concatenate((Hx, zeros), axis=1),
    np.concatenate((zeros, Hz), axis=1)
))
print("Combined Parity-Check Matrix H0:\n", H0)


Combined Parity-Check Matrix H0:
 [[1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1.]]


In [6]:
def get_stabilizers(H_sub, stab):
    """
    Generate stabilizer strings from a parity-check matrix.
    
    Parameters:
        H_sub (ndarray): Submatrix of Hx or Hz.
        stab (str): Type of stabilizer ('X' or 'Z').
        
    Returns:
        all_stab (list): List of stabilizer strings.
    """
    all_stab = []
    for n in range(len(H_sub)):
        curr_str = ''
        row = H_sub[n]
        for m in row:
            if m:
                curr_str += stab
            else:
                curr_str += 'I'
        print(curr_str)
        all_stab.append(curr_str)
    return all_stab

# Display X stabilizers
print("X Stabilizers:")
x_stabilizers = get_stabilizers(Hx, 'X')

# Display Z stabilizers
print("\nZ Stabilizers:")
z_stabilizers = get_stabilizers(Hz, 'Z')


X Stabilizers:
XIIXIXXXIXII
XXIIXIIXXIXI
IXXIIXIIXXIX
XIXXIIXIIXXI
IXIXXIIXIIXX
IIXIXXXIXIIX

Z Stabilizers:
ZIIZIZZZIZII
ZZIIZIIZZIZI
IZZIIZIIZZIZ
ZIZZIIZIIZZI
IZIZZIIZIIZZ
IIZIZZZIZIIZ


In [7]:
# %%
def make_stabilizer_circ_1(Hx, Hz, qc, noise_inject=None):
    """
    Generate a stabilizer circuit given Hx and Hz.

    Parameters:
        Hx (ndarray): Parity-check matrix for X errors.
        Hz (ndarray): Parity-check matrix for Z errors.
        qc (QuantumCircuit): The quantum circuit to modify.
        noise_inject (list, optional): Functions to inject noise.

    Returns:
        qc (QuantumCircuit): Modified quantum circuit.
    """
    qc.barrier()
    data = qc.qregs[0]      # Data qubits
    ancilla = qc.qregs[1]   # Ancilla qubits

    z_offset = Hx.shape[0]  # Offset for Z stabilizers in ancilla

    # Apply X stabilizers
    for i in range(Hx.shape[0]):
        row = Hx[i]
        for j, val in enumerate(row):
            if val:
                qc.cx(data[j], ancilla[i])
                print(f"Added X stabilizer from data qubit {j} to ancilla qubit {i}")
                qc.barrier()

    # Apply Z stabilizers
    for i in range(Hz.shape[0]):
        row = Hz[i]
        for j, val in enumerate(row):
            if val:
                qc.cz(data[j], ancilla[i + z_offset])
                print(f"Added Z stabilizer from data qubit {j} to ancilla qubit {i + z_offset}")
                qc.barrier()

    # Measure ancilla qubits to obtain syndromes
    for i in range(ancilla.size):
        qc.measure(ancilla[i], qc.cregs[0][i])

    return qc

In [8]:
def get_decoder(p):
    """
    Initialize the BP-OSD decoder.
    
    Parameters:
        p (float): Error rate (expected as Python float).
        
    Returns:
        bp_osd (BpOsdDecoder): Configured BP-OSD decoder.
    """
    bp_osd = BpOsdDecoder(
        H0,
        error_rate=float(p),  # Convert numpy.float64 to Python float
        bp_method='product_sum',
        max_iter=15,
        schedule='serial',
        osd_method='osd_cs',  # Use 'osd_cs' for efficient decoding
        osd_order=2
    )
    return bp_osd


In [9]:
# Create a fully connected coupling map for 24 qubits
num_qubits = 24
coupling_list = [(i, j) for i in range(num_qubits) for j in range(num_qubits) if i != j]
coupling_map = CouplingMap(couplinglist=coupling_list)

In [10]:
def inject_bit_flip_errors(circuit, qubits, error_probabilities):
    """
    Injects bit-flip (X) errors into specified qubits based on their respective probabilities.

    Parameters:
        circuit (QuantumCircuit): The quantum circuit to modify.
        qubits (list or QuantumRegister): The qubits to inject errors into.
        error_probabilities (ndarray): Array of error probabilities for each qubit.

    Returns:
        QuantumCircuit: Modified quantum circuit with errors injected.
    """
    for qubit, p in zip(qubits, error_probabilities):
        if np.random.rand() < p:
            circuit.x(qubit)
            print(f"Injected X error on qubit {qubit}")
    return circuit


In [11]:
def inject_phase_flip_errors(circuit, qubits, error_probabilities):
    for qubit, p in zip(qubits, error_probabilities):
        if np.random.rand() < p:
            circuit.z(qubit)
            print(f"Injected Z error on qubit {qubit}")
    return circuit


In [12]:
def create_circuit_with_noise(num_qubits, error_probabilities):
    """
    Create a quantum circuit with specified noise on each qubit.
    
    Parameters:
        num_qubits (int): Number of qubits.
        error_probabilities (ndarray): Array of error probabilities for each qubit.
        
    Returns:
        circuit (QuantumCircuit): Quantum circuit with noise.
    """
    qreg = QuantumRegister(num_qubits, 'q')
    creg = ClassicalRegister(num_qubits, 'c')
    circuit = QuantumCircuit(qreg, creg)
    
    for qubit in range(num_qubits):
        # Apply bit-flip error based on probability
        if np.random.rand() < error_probabilities[qubit]:
            circuit.x(qreg[qubit])
    
    return circuit
    
def some_condition_based_on_measurements(counts, logical_qubit_indices=None):
    """
    Determines if a logical error has occurred based on measurement results.

    Parameters:
        counts (dict): Measurement results.
        logical_qubit_indices (list, optional): Indices of qubits that are logical qubits.
                                                If None, all measured bits are treated as logical.
    Returns:
        bool: True if a logical error is detected, False otherwise.
    """
    # Grab the single-shot bitstring (e.g. '01011…')
    measured_bits = list(counts.keys())[0]
    # If no indices passed, assume every bit in that string is a logical qubit
    if logical_qubit_indices is None:
        logical_qubit_indices = list(range(len(measured_bits)))
    # Pull out only those bits
    logical_bits = [measured_bits[i] for i in logical_qubit_indices]
    # Logical error if any logical bit is '1'
    return any(bit == '1' for bit in logical_bits)

In [13]:
def construct_encoding_circuit():
    """
    Constructs the encoding circuit for logical qubits.

    Returns:
        QuantumCircuit: The encoding circuit without measurements.
    """
    data_qubits = QuantumRegister(Hx.shape[1], 'data')
    ancilla_qubits = QuantumRegister(Hx.shape[0] + Hz.shape[0], 'ancilla')
    creg = ClassicalRegister(Hx.shape[0] + Hz.shape[0], 'c')
    encoding_circuit = QuantumCircuit(data_qubits, ancilla_qubits, creg)
    
    # Example encoding operations (modify based on your specific code)
    # encoding_circuit.h(data_qubits[0])
    # encoding_circuit.cx(data_qubits[0], data_qubits[1])
    # Add your encoding gates here
    
    return encoding_circuit


In [14]:
def calculate_logical_error_rate(p, num_qubits, times, noise_factor, return_circuit=False):
    logical_error_num = 0
    # Use statevector simulator
    simulator = AerSimulator(method='statevector')

    # Initialize the decoder
    bp_osd = get_decoder(p=p)

    data_qubits = Hx.shape[1]
    ancilla_qubits = Hx.shape[0] + Hz.shape[0]

    assert num_qubits == data_qubits + ancilla_qubits, \
        f"num_qubits should be {data_qubits + ancilla_qubits} (data + ancilla), but got {num_qubits}"

    # (You can remove coupling_map entirely if it's no longer used)
    coupling_list = [(i, j) for i in range(num_qubits) for j in range(num_qubits) if i != j]
    coupling_map = CouplingMap(couplinglist=coupling_list)

    # Build encoding / decoding
    encoding_circuit = construct_encoding_circuit()
    decoding_circuit = encoding_circuit.inverse()

    # Transpile the decoder _without_ a coupling_map
    decoding_circuit_transpiled = transpile(decoding_circuit, simulator)

    circuit_to_return = None

    for run in range(times):
        print(f"\nRunning simulation {run + 1}/{times}")

        # Build fresh circuit
        data_reg = QuantumRegister(data_qubits, 'data')
        ancilla_reg = QuantumRegister(ancilla_qubits, 'ancilla')
        creg = ClassicalRegister(ancilla_qubits, 'c')
        circuit = QuantumCircuit(data_reg, ancilla_reg, creg)

        # Encode
        circuit.compose(encoding_circuit, inplace=True)

        # Inject randomized noise
        scaled_error_probabilities = p + noise_factor * np.random.uniform(-1, 1, size=data_qubits)
        unique_error_probabilities = np.clip(scaled_error_probabilities, 0, 1)
        print("Unique Error Probabilities per Data Qubit:", unique_error_probabilities)
        circuit = inject_bit_flip_errors(circuit, data_reg, unique_error_probabilities)
        circuit = inject_phase_flip_errors(circuit, data_reg, unique_error_probabilities)

        # Measure stabilizers
        circuit = make_stabilizer_circ_1(Hx, Hz, circuit)

        # (Optional) capture first run’s full circuit
        if return_circuit and run == 0:
            circuit.compose(decoding_circuit_transpiled, qubits=circuit.qubits, inplace=True)
            for i in range(data_qubits):
                circuit.measure(data_reg[i], creg[i])
            circuit_to_return = circuit.copy()

        # Transpile & run for syndrome (no coupling_map)
        transpiled_circuit = transpile(circuit, simulator)
        job = simulator.run(transpiled_circuit, shots=1)
        result = job.result()
        counts = result.get_counts()

        # Decode
        measured_bits = list(counts.keys())[0]
        syndrome = np.array([int(bit) for bit in measured_bits[::-1]])
        print(f"Syndrome: {syndrome}")
        decoding = bp_osd.decode(syndrome)
        print(f"Decoding: {decoding}")

        # Apply correction
        for i in range(data_qubits):
            if decoding[i] != 0:
                circuit.z(data_reg[i])
            if decoding[i + ancilla_qubits // 2] != 0:
                circuit.x(data_reg[i])

        # Inverse‐encode & measure logicals
        circuit.compose(decoding_circuit_transpiled, qubits=circuit.qubits, inplace=True)
        for i in range(data_qubits):
            circuit.measure(data_reg[i], creg[i])

        # Transpile & run measurement (no coupling_map)
        transpiled_circuit_measure = transpile(circuit, simulator)
        job_measure = simulator.run(transpiled_circuit_measure, shots=1)
        counts_measure = job_measure.result().get_counts()

        # Check for logical error
        if some_condition_based_on_measurements(counts_measure,
                                                logical_qubit_indices=list(range(data_qubits))):
            logical_error_num += 1

    logical_error_rate = logical_error_num / times

    if return_circuit:
        return logical_error_rate, circuit_to_return
    else:
        return logical_error_rate


In [15]:
from qiskit.transpiler import CouplingMap

# Verify the coupling map
print("Coupling Map is fully connected:")

# Check that every qubit is connected to every other qubit
is_fully_connected = True
for i in range(num_qubits):
    connected = set([j for (a, j) in coupling_map.get_edges() if a == i])
    if len(connected) != num_qubits - 1:
        is_fully_connected = False
        print(f"Qubit {i} is connected to {len(connected)} qubits.")
        break

if is_fully_connected:
    print("All qubits are fully connected.")
else:
    print("Coupling map is not fully connected.")


Coupling Map is fully connected:
All qubits are fully connected.


In [ ]:
np.random.seed(42)  # Sets the seed for NumPy's random number generator

# Define simulation parameters
physical_error_rates = np.linspace(0.00, 0.5, 15)
logical_error_rates = []
error_bars = []  # To store asymmetric error bars
noise_factor = 0.0  # Set to 0 for consistent error probabilities
num_qubits = 24
times = 50  # Increased number of runs for statistical significance

for p in physical_error_rates:
    print(f"\nProcessing physical error rate: {p:.4f}")
    # Collect logical error outcomes for 'times' runs
    logical_errors = []
    for _ in range(times):
        logical_error = calculate_logical_error_rate(
            p=p, 
            num_qubits=num_qubits, 
            times=1,  # Each call runs one simulation
            noise_factor=noise_factor
        )
        logical_errors.append(logical_error)
    # Compute mean and standard error
    mean_error = np.mean(logical_errors)
    std_error = np.sqrt(mean_error * (1 - mean_error) / times)
    logical_error_rates.append(mean_error)
    # Calculate asymmetric error bars
    lower_error = min(mean_error, std_error)
    upper_error = min(1 - mean_error, std_error)
    error_bars.append([lower_error, upper_error])

# Extract lower and upper errors for plotting
lower_errors = [eb[0] for eb in error_bars]
upper_errors = [eb[1] for eb in error_bars]

# Plotting the results with error bars on a linear scale
plt.figure(figsize=(8, 6))
plt.errorbar(
    physical_error_rates,
    logical_error_rates,
    yerr=[lower_errors, upper_errors],
    fmt='o-',
    color='b',
    ecolor='r',
    capsize=5,
    label='Logical vs Physical Error Rate'
)
plt.xlabel('Physical Error Rate')
plt.ylabel('Logical Error Rate')
plt.title('Logical Error Rate vs Physical Error Rate with Error Bars (Linear Scale)')
plt.legend()
plt.grid(True)
plt.yscale('linear')  # Change to 'linear' scale
plt.show()

In [ ]:
# Plotting the results with error bars on a linear scale
plt.figure(figsize=(8, 6))
plt.errorbar(
    physical_error_rates,
    logical_error_rates,
    yerr=[lower_errors, upper_errors],
    fmt='o-',
    color='b',
    ecolor='r',
    capsize=5,
    label='Logical vs Physical Error Rate'
)
plt.xlabel('Physical Error Rate')
plt.ylabel('Logical Error Rate')
plt.title('Logical Error Rate vs Physical Error Rate with Error Bars (Linear Scale)')
plt.legend()
plt.grid(True)
plt.yscale('linear')  # Change to 'linear' scale        
plt.show()

In [ ]:
# Define simulation parameters
physical_error_rates = np.linspace(0.00, 0.5, 11)
logical_error_rates = []
noise_factor = 0.1  # Controls how much variation there is
num_qubits = 24
times = 1  # Set to 1 to capture the circuit from a single run

# Choose a specific physical error rate for visualization, e.g., p=0.1
p = 0.1

# Call the function with return_circuit=True
logical_error, complete_circuit = calculate_logical_error_rate(p, num_qubits, times, noise_factor, return_circuit=True)

# Print the logical error rate
print(f"\nLogical Error Rate at p={p}: {logical_error}")


In [ ]:
%matplotlib inline

# Draw the complete circuit using Matplotlib (graphical representation)
complete_circuit.draw(output='mpl', fold=300)
plt.show()
# Check if the circuit is being created correctly
print(complete_circuit)


# Alternatively, for a text-based representation:
#print(complete_circuit.draw(output='text'))


In [ ]:
import numpy as np

# Pauli matrices
I_mat = np.array([[1,0],[0,1]], dtype=complex)
X_mat = np.array([[0,1],[1,0]], dtype=complex)
Y_mat = np.array([[0,-1j],[1j,0]], dtype=complex)
Z_mat = np.array([[1,0],[0,-1]], dtype=complex)

paulis_mat = {'I': I_mat, 'X': X_mat, 'Y': Y_mat, 'Z': Z_mat}

def op_to_mat(op):
    return np.kron(np.kron(paulis_mat[op[0]], paulis_mat[op[1]]), paulis_mat[op[2]])

# Given final cosets with phases:
coset1 = [(1,'III'),(1,'XXX'),(1,'XZY'),(1,'IYZ')]
coset2 = [(1,'IXX'),(1,'XII'),(1,'XYZ'),(1,'IZY')]
coset3 = [(1,'YIZ'),(1,'ZXY'),(-1,'ZZX'),(1,'YYI')]
coset4 = [(1,'YXY'),(-1,'ZIZ'),(-1,'ZYI'),(-1,'YZX')]

cosets = [coset1, coset2, coset3, coset4]

projectors = []
for i,coset in enumerate(cosets):
    sum_mat = np.zeros((8,8), dtype=complex)
    for (ph, opn) in coset:
        sum_mat += ph * op_to_mat(opn)
    P = (1/4)*sum_mat
    projectors.append(P)

def is_hermitian(M):
    return np.allclose(M, M.conjugate().T, atol=1e-12)

def is_idempotent(M):
    return np.allclose(M@M, M, atol=1e-12)

# Check properties
print("Checking properties of the four projectors:\n")
for i,P in enumerate(projectors):
    herm = is_hermitian(P)
    idem = is_idempotent(P)
    print(f"P^{i+1} Hermitian: {herm}, Idempotent: {idem}")

# Check orthogonality
for i in range(4):
    for j in range(i+1,4):
        prod = projectors[i]@projectors[j]
        if np.allclose(prod, 0, atol=1e-12):
            print(f"P^{i+1} and P^{j+1} are orthogonal.")
        else:
            print(f"WARNING: P^{i+1} and P^{j+1} are NOT orthogonal.")

# Check completeness
P_sum = sum(projectors)
if np.allclose(P_sum, np.eye(8), atol=1e-12):
    print("Sum of all projectors equals I_8.")
else:
    print("WARNING: Sum of all projectors does not equal I_8.")

